# Laboratorio de estimación no-paramétrica
Procesamiento Adaptativo de Señales Aleatorias

© 2019 Tomas A. González Orlando

© 2019 Lucero G. Fernandez

# Ejercicio 1



## Preparativos
Debe guardarse el archivo readligo.py en el directorio correspondiente para poder leer los datos correctamente.

### Clonamos el repositorio de Github

In [1]:
#https://medium.com/@navan0/how-to-push-files-into-github-from-google-colab-379fd0077aa8
!git clone https://github.com/LuFernandez/PASA.git

Cloning into 'PASA'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 564 (delta 13), reused 20 (delta 3), pack-reused 520
Receiving objects: 100% (564/564), 83.79 MiB | 33.03 MiB/s, done.
Resolving deltas: 100% (267/267), done.


In [2]:
!git init
!git config -- global user.email “togonzalez@itba.edu.ar”
!git config -- global user.name “taomasgonzalez”

Initialized empty Git repository in /content/.git/
error: key does not contain a section: global
error: key does not contain a section: global


### Imports necesarios a lo largo del ejercicio

In [3]:
import h5py as h5

import scipy
from scipy import signal as scsig
from scipy.interpolate import interp1d
from scipy import fftpack as scfft

import numpy as np
from numpy.lib import scimath

import matplotlib.pyplot as plt
import readligo as rl
from sklearn.metrics import mean_squared_error
import math

import matplotlib.mlab as mlab

ModuleNotFoundError: ignored

## Lectura de datos

In [0]:
def read_data(path, name):
  strain, time, chan_dict = rl.loaddata(path, name)
  dt = time[1] - time[0]
  times = dt*np.array([i for i in range(len(strain))])
  return (strain, times, chan_dict, dt)

fs = 4096
folder_path = "PASA/LAB1/LIGO/"
# leemos los datos de Livingston
strain_l1, time_l1, chan_dict_l1, dt_l1 = read_data(path=folder_path+'L-L1_LOSC_4_V2-1126259446-32.hdf5', name='L1')
# leemos los datos de Hanford
strain_h1, time_h1, chan_dict_h1, dt_h1 = read_data(path=folder_path+'H-H1_LOSC_4_V2-1126259446-32.hdf5', name='H1')
# strain_h1, time_h1, chan_dict_h1 = rl.loaddata('H-H1_LOSC_4_V2-1126259446-32.hdf5', 'L1')


## Visualización de datos

In [0]:
plt.plot(time_l1, strain_h1)
plt.plot(time_h1, strain_l1)
plt.grid()
plt.show()

## a) Estimación del espectro de potencia del ruido mediante el uso de periodogramas

### Implementación del periodograma

In [0]:
def periodogram(fs, data, figure_number, x_lim, title, plot=True):
  # cantidad de muestras
  N = len(data)
  z = scfft.fft(x=data, n=N)
  R = z*np.conj(z)
  freqs = np.fft.fftfreq(N)
  xf = np.linspace(0, fs/2, N//2)    #frecuencias
  if plot:
    plt.loglog(xf, 2/N * np.abs(R[:N//2]))
    plt.xlim(x_lim)
    plt.grid()
    plt.title(title)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power Spectral Density Rx(f)")
  
  return (xf, R[:N//2])

### Aplicado a Livingston
Hacemos el periodograma para los datos de Livingston en su totalidad, y divididimos en sectores para calcular el error y corroborar estacionariedad.

In [0]:
p_f_l1_tot, p_Pxx_l1_tot = periodogram(fs=fs, data=strain_l1, figure_number=1, x_lim=(10, 2000), title="Periodograma L1")
plt.show()
p_f_l1_first, p_Pxx_l1_first = periodogram(fs=fs, data=strain_l1[:len(strain_l1)//3], figure_number=1, x_lim=(10, 2000), title="Periodograma L1, primeros 10 segundos")
plt.show()
p_f_l1_last, p_Pxx_l1_last = periodogram(fs=fs, data=strain_l1[int(len(strain_l1)*2/3):], figure_number=1, x_lim=(10, 2000), title="Periodograma L1, últimos 10 segundos")
plt.show()

In [0]:
 rmse = np.square(np.abs(np.subtract(p_Pxx_l1_first,p_Pxx_l1_last))).mean() 
 print("La raíz del error cuadrático medio o RMSE (Rooted Mean Square Error) es:", rmse,
       "\npor lo que se puede decir que el periodograma de un tramo de los datos es representativo de todo el ensamble.")

### Aplicado a Hanford
Hacemos el periodograma para los datos de Hanford en su totalidad, y divididimos en secotres para calcular el error y corroborar estacionariedad.

In [0]:
p_f_h1_tot, Pxx_h1_tot = periodogram(fs=fs, data=strain_h1, figure_number=1, x_lim=(10, 2000), title="Periodograma H1")
plt.show()
p_f_h1_first, p_Pxx_h1_first = periodogram(fs=fs, data=strain_h1[:len(strain_h1)//3], figure_number=2, x_lim=(10, 2000), title="Periodograma H1, primeros 10 segundos")
plt.show()
p_f_h1_last, p_Pxx_h1_last = periodogram(fs=fs, data=strain_h1[2*len(strain_l1)//3:], figure_number=3, x_lim=(10, 2000), title="Periodograma H1, últimos 10 segundos")
plt.show()

In [0]:
rmse = np.square(np.abs(np.subtract(p_Pxx_h1_first,p_Pxx_h1_last))).mean() 
print("La raíz del error cuadrático medio o RMSE (Rooted Mean Square Error) es:", rmse,
      "\npor lo que se puede decir que el periodograma de un tramo de los datos es representativo de todo el ensamble.")

###Superponemos gráficos

In [0]:
p_f_h1_first, p_Pxx_h1_first = periodogram(fs=fs, data=strain_h1[:len(strain_h1)//3], figure_number=2, x_lim=(10, 2000), title="Periodograma H1, primeros 10 segundos")
p_f_l1_first, p_Pxx_l1_first = periodogram(fs=fs, data=strain_l1[:len(strain_l1)//3], figure_number=1, x_lim=(10, 2000), title="Periodograma L1, primeros 10 segundos")
plt.legend(['H1','L1'])
plt.grid()
plt.show()

## b) Estimación del espectro de potencia del ruido mediante el uso del método de Blackman-Tukey

### Implementación del método de Blackman-Tukey

Definimos cómo crear la ventana necesaria:

In [0]:
def create_b_tukey_window(N, M, L):
  delta_w = 2*np.pi/N* (2*M+1) 
  l = np.arange(1,L+1)
  window = np.sin(delta_w*l / 2)  / (np.pi * l)                 
  window = np.append( np.flip(window), np.append([delta_w/2/np.pi] , window) )
  return window

Definimos cómo estimar la autocorrelación de una señal ruidosa:

In [0]:
def autocorrelation_estimation(data, L, N):
  # Formamos el vector r_x con todos l de 0 a L 
  if data.dtype is not complex:
    data = data.astype(complex)
    
  # complex conjugate of the first argument is used for the calculation of the dot product.
  r_x = np.array([np.vdot(data[0:N-l]/N, data[l:N]) for l in range(0, L+1)])
  # terminamos de completar r_x con los valores de -L a -1:
  r_x = np.append( np.flip( np.conj(r_x[1:]) ) , r_x )
  return r_x

Finalmente, implementamos el método:

In [0]:
#frac_N_l = N/l
def blackman_tukey(fs, data, figure_number, M, x_lim, title='', N=4096, frac_N_l=5, plot=True):
  # cantidad de muestras N
  # Se recomienda N mayor a 50, y |l| ≤ N/4 (Box & Jenkins, 1976). 
  L = N//frac_N_l
  if N <= 50:
    print('Se recomienda elegir una cantidad de muestras mayor a 50! (Box & Jenkins, 1976)')
  if L > N//4:
    print('Se recomienda elegir |l|<=N/4! (Box & Jenkins, 1976)')
    
  r_xw = autocorrelation_estimation(data=data, L=L, N=N) * create_b_tukey_window(N, M, L)

  z = abs(scfft.fft(x=r_xw, n=len(r_xw)))
  
  freqs = np.fft.fftfreq(n=len(r_xw), d=1/fs)
  if plot: 
    plt.loglog(freqs, z)
    plt.xlim(x_lim)
    plt.grid()
    plt.title(title)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power Spectral Density Rx(f)")
  return (freqs, z)

### Aplicado a los datos de Livingston

In [0]:
b_f_l1_tot, b_Pxx_l1_tot = blackman_tukey(fs=fs, data=strain_l1, figure_number=1, x_lim=(10,2000), title='Strain L1', M=4, N=4096*10, frac_N_l=16)
plt.show()
b_f_l1_first, b_Pxx_l1_first = blackman_tukey(fs=fs, data=strain_l1[:len(strain_l1)//3], figure_number=1, x_lim=(10,2000), title='Strain L1', M=4, N=4096*3, frac_N_l=16)
plt.show()
b_f_l1_last, b_Pxx_l1_last = blackman_tukey(fs=fs, data=strain_l1[2*len(strain_l1)//3:], figure_number=1, x_lim=(10,2000), title='Strain L1', M=4, N=4096*3, frac_N_l=8)
plt.show()

### Aplicado a los datos de Hanford

In [0]:
b_f_h1_tot, b_Pxx_h1_tot = blackman_tukey(fs=fs, data=strain_h1, figure_number=1, x_lim=(10,2000), title='Strain H1', M=20,N=4096*5, frac_N_l=5)
plt.show()
b_f_h1_first, b_Pxx_h1_first = blackman_tukey(fs=fs, data=strain_h1, figure_number=1, x_lim=(10,2000), title='Strain H1', M=20,N=4096*2, frac_N_l=5)
plt.show()
b_f_h1_last, b_Pxx_h1_last = blackman_tukey(fs=fs, data=strain_h1, figure_number=1, x_lim=(10,2000), title='Strain H1', M=20,N=4096*3, frac_N_l=5)
plt.show()

###Superponemos gráficos

In [0]:
b_f_h1_first, b_Pxx_h1_first = blackman_tukey(fs=fs, data=strain_h1, figure_number=1, x_lim=(10,2000), title='Strain H1', M=20,N=4096*2, frac_N_l=5)
b_f_l1_first, b_Pxx_l1_first = blackman_tukey(fs=fs, data=strain_l1[:len(strain_l1)//3], figure_number=1, x_lim=(10,2000), title='Strain L1', M=4, N=4096*3, frac_N_l=16)
plt.legend(['H1','L1'])
plt.grid()
plt.show()

## c) Estimación del espectro de potencia del ruido mediante el uso del método de Welch-Bartlett

### Implementación del método de Welch-Bartlett

En la implementación de Welch Bartlett, se utiliza una ventana de tipo Hanning para reducir el bias. Probándose con una ventana cuadrada, se observó un bias notable (las frecuencias altas se veían atenuadas con respecto al espectro correcto).

In [0]:
#Inspirado en https://stackoverflow.com/questions/46882312/python-implementation-of-bartlett-periodogram

#nperseg : L
def welch_barlett(data, fs,  D=0, nperseg=512, nfft=512, title='',x_lim=(10,2000), plot=True):
    if nfft % 2:
      numFreqs = (nfft + 1)//2
    else:
      numFreqs = nfft//2 + 1
      
    x = mlab.stride_windows(data, nfft, 0, axis=0)
    window = np.ones(nfft, x.dtype)
    window = np.hanning(len(np.ones(nfft, x.dtype)))*np.ones(nfft, x.dtype)
    x = x * window.reshape((-1, 1))
    x = np.fft.fft(x, n=nfft, axis=0)[:numFreqs, :]
    freqs = np.fft.fftfreq(nfft, 1/fs)[:numFreqs]
    freqs[len(freqs)-1] = -freqs[len(freqs)-1]
    x = np.conj(x) * x
    
    x /= fs
    x /= (np.abs(window)**2).sum()
    x = np.transpose(x).sum(axis=0)/len(x[1])
    if plot: 
      plt.loglog(freqs, x)
      plt.xlim(x_lim)
      plt.grid()
      plt.title(title)
      plt.xlabel("Frequency (Hz)")
      plt.ylabel("Power Spectral Density Rx(f)")    
    
    return freqs, x 


### Aplicado a los datos de Livingston

In [0]:
nperseg = 4096
nfft = nperseg
w_f_l1_tot, w_Pxx_l1_tot = welch_barlett(data=strain_l1, fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett L1',x_lim=(10,2000), plot=True)
plt.show()
w_f_l1_first, w_Pxx_l1_first = welch_barlett(data=strain_l1[:len(strain_l1)//3], fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett L1',x_lim=(10,2000), plot=True)
plt.show()
w_f_l1_last, w_Pxx_l1_last = welch_barlett(data=strain_l1[2*len(strain_l1)//3:], fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett L1',x_lim=(10,2000), plot=True)
plt.show()

### Aplicado a los datos de Hanford

In [0]:
w_f_h1_tot, w_Pxx_h1_tot = welch_barlett(data=strain_h1, fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett H1',x_lim=(10,2000), plot=True)
plt.show()
w_f_h1_first, w_Pxx_h1_first = welch_barlett(data=strain_h1[:len(strain_h1)//3], fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett H1',x_lim=(10,2000), plot=True)
plt.show()
w_f_h1_last, w_Pxx_h1_last = welch_barlett(data=strain_h1[2*len(strain_h1)//3:], fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett H1',x_lim=(10,2000), plot=True)
plt.show()

###Superponemos gráficos

In [0]:
w_f_h1_first, w_Pxx_h1_first = welch_barlett(data=strain_h1[:len(strain_h1)//3], fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett H1',x_lim=(10,2000), plot=True)
w_f_l1_first, w_Pxx_l1_first = welch_barlett(data=strain_l1[:len(strain_l1)//3], fs=fs, nperseg=nperseg, nfft=nfft, title='Welch-Barlett L1',x_lim=(10,2000), plot=True)
plt.legend(['H1','L1'])
plt.grid()
plt.show()

## d) Estimación del espectro de potencia del ruido mediante el uso del enfoque multitaper con funciones DPSS

### Implementación del método multitaper con funciones DPSS

In [0]:
# para la matemática:
# https://ccrma.stanford.edu/~jos/sasp/Slepian_DPSS_Window.html
# para las ventanas:
# https://scipy.github.io/devdocs/generated/scipy.signal.windows.dpss.html
def multitaper_dpss(data, fs, windows, x_lim=(10,2000), title='', plot=True):
  # tomo a x_lim[1] como el bandwith!!!
  R_i = abs( np.fft.fft( a=np.multiply(data,windows) ) ) ** 2 / len(data)
  freqs = np.fft.fftfreq( n=len(data), d=1/fs )
  R = np.sum(R_i, axis=0) / len(windows)
  if plot: 
    plt.loglog(freqs, 2/len(data) * R)
    plt.xlim(x_lim)
    plt.grid()
    plt.title(title)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power Spectral Density Rx(f)")
  return freqs, R

### Aplicado a los datos de Livingston

Preparamos las ventanas slepianas con las que vamos a trabajar sobre los datos.

In [0]:
# sym bool, optional
# When True (default), generates a symmetric window, for use in filter design. When False, generates a periodic window, for use in spectral analysis.

#tomamos bandwith = 2e3
sequences = scsig.windows.dpss(M=len(strain_l1), NW=4096//2, Kmax=4, sym=False)
sequences_short = scsig.windows.dpss(M=len(strain_l1)//3, NW=4096//2, Kmax=4, sym=False)


In [0]:
d_f_l1_tot, f_Pxx_l1_tot = multitaper_dpss(data=strain_l1, fs=fs, windows=sequences, x_lim=(10,2000), title='Multitaper L1', plot=True)
plt.show()
d_f_l1_first, f_Pxx_l1_first = multitaper_dpss(data=strain_l1[:len(strain_l1)//3], fs=fs, windows=sequences_short, x_lim=(10,2000), title='Multitaper L1, primeros 10 segundos', plot=True)
plt.show()
d_f_l1_last, f_Pxx_l1_last = multitaper_dpss(data=strain_l1[2*len(strain_l1)//3+1:], fs=fs, windows=sequences_short, x_lim=(10,2000), title='Multitaper L1, últimos 10 segundos', plot=True)
plt.show()

### Aplicado a los datos de Hanford

Preparamos las ventanas slepianas con las que vamos a trabajar sobre los datos.

In [0]:
# sym bool, optional
# When True (default), generates a symmetric window, for use in filter design. When False, generates a periodic window, for use in spectral analysis.

#tomamos bandwith = 2e3
sequences = scsig.windows.dpss(M=len(strain_h1), NW=4096//2, Kmax=6, sym=False)
sequences_short = scsig.windows.dpss(M=len(strain_h1)//3, NW=4096//2, Kmax=7, sym=False)

In [0]:
d_f_h1_tot, d_Pxx_h1_tot = multitaper_dpss(data=strain_h1, fs=fs, windows=sequences, x_lim=(10,2000), title='Multitaper H1', plot=True)
plt.show()
d_f_h1_first, d_Pxx_h1_first = multitaper_dpss(data=strain_h1[:len(strain_h1)//3], fs=fs, windows=sequences_short, x_lim=(10,2000), title='Multitaper H1, primeros 10 segundos', plot=True)
plt.show()
d_f_h1_last, d_Pxx_h1_last = multitaper_dpss(data=strain_h1[2*len(strain_h1)//3+1:], fs=fs, windows=sequences_short, x_lim=(10,2000), title='Multitaper H1, últimos 10 segundos', plot=True)
plt.show()

###Superponemos gráficos

In [0]:
d_f_h1_first, d_Pxx_h1_first = multitaper_dpss(data=strain_h1[:len(strain_h1)//3], fs=fs, windows=sequences_short, x_lim=(10,2000), title='Multitaper H1, primeros 10 segundos', plot=True)
d_f_l1_first, f_Pxx_l1_first = multitaper_dpss(data=strain_l1[:len(strain_l1)//3], fs=fs, windows=sequences_short, x_lim=(10,2000), title='Multitaper L1, primeros 10 segundos', plot=True)
plt.legend(['H1','L1'])
plt.grid()
plt.show()

La mejor estimación fue obtenida con el método de Welch-Bartlett, además fue sencillo encontrar parámetros que se ajustaran al espectro. Por otro lado, la obtenida con Blackman-Tukey fue poco satisfactoria ya que se notó la presencia de bias, posiblemente debido a la ventana elegida. El periodograma, por su parte, fue sencillo de implementar, pero la alta varianza de la estimación lo hace un método no muy utilizado, ya que es preferible usar Welch-Bartlett. Por último, con la estimación multitaper se obtuvieron resultados aceptables, sacrificando resolución espectral, debido a las ventanas usadas.

#Ejercicio 2

## Inversa del espectro de potencia del ruido


### Implementación el filtro inversor

In [0]:
psd_l1 = interp1d(w_f_l1_tot, w_Pxx_l1_tot)
psd_h1 = interp1d(w_f_h1_tot, w_Pxx_h1_tot)
                  
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)
    hf = np.fft.rfft(strain)
    white_hf = hf / np.sqrt(interp_psd(freqs) /dt/2.)
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

strain_h1_whiten = whiten(strain_h1, psd_h1, dt_h1)
strain_l1_whiten = whiten(strain_l1, psd_l1, dt_l1)
bb, ab = scsig.butter(4, [20*2./fs, 300*2./fs], btype='band')
strain_H1_whitenbp = scsig.filtfilt(bb, ab, strain_h1_whiten)
strain_L1_whitenbp = scsig.filtfilt(bb, ab, strain_l1_whiten)

## Señal gravitatoria en el momento del evento

In [0]:
strain_L1_shift = -np.roll(strain_L1_whitenbp, int(0.007*fs))

plt.figure()
plt.plot(np.arange(66750,67500)*dt_l1, strain_L1_shift[66750:67500],'g',label='L1 strain')
plt.plot(np.arange(66750,67500)*dt_h1, strain_H1_whitenbp[66750:67500],'r',label='H1 strain')
plt.xlabel('Tiempo t')
plt.ylabel('Señal a la salida del filtro ')
plt.legend(loc='lower left')
plt.title('Señal gravitatoria en el momento del evento')


from IPython.display import Audio
Audio(strain_L1_whitenbp[66750:67500],rate=4096)

## Buscamos el time shift correspondiente

Debido a que estamos trabajando con ondas cuyas velocidades corresponden a velocidades relativistas, sabemos que los eventos no sucederán simultáneamente en cada sistema. 
Buscamos encontrar entonces el corrimiento en tiempo que deberemos realizar en los datos de uno de los dos sistemas para poder verificar la correspondencia entre eventos.
Para solucionar este problema, hallamos las correlaciones cruzadas entre los datos de Hanford y los de Livingston. 
Pondremos una cota para los tiempos de shifting que podremos encontrar para así reducir el tiempo de cálculo.
Dado que la distancia entre Hanford y Livingston es de  3030.13 km, (https://astronomy.stackexchange.com/questions/13704/gravitational-wave-detection-time-difference-between-ligo-livingston-and-ligo-ha) y la velocidad de la luz es de 299792458 (km/s), entonces el máximo shifting en tiempo que se podrá tener entre las dos señales es de 10(ms). 


### Implementación de la autocorrelación y obtención del time shift

In [0]:
# https://stackoverflow.com/a/4688875
def get_time_shift(data1, data2):
  time_shift = np.argmax(np.abs(scsig.correlate(data1,data2, 'same')[:int(10e-3/dt_l1)]))
  return time_shift

time_shift = get_time_shift(strain_L1_whitenbp[:len(strain_L1_whitenbp)//3], strain_H1_whitenbp[:len(strain_H1_whitenbp)//3])
print('Time shift: %0.2f milisegundos' % float(time_shift*dt_l1*1e3) )

#Ejercicio 3

## Mejora de la estimación del espectro de potencia


Según lo que menciona el paper https://arxiv.org/pdf/1604.06211.pdf, tanto la señal de Livingston $S_L$, como la de Hanford $S_H$ se pueden expresar como:

$S_L(t) = G(t) + n_L(t) + F_L(t)$

$S_H = (±)G(t ± τ ) + n_H(t) + F_H(t)$

con $G(t)$ el evento observado en el detector $L$ y observado, (posiblemente invertido) en el detector $H$ en un tiempo shifteado por $±τ$. Además, $n_H(t)$, $n_L(t)$ son las componentes de ruido de banda ancha y $F_L(t)$, $F_H(t)$ se refieren a los efectos de banda angosta que pueden ser tiempo-dependientes.





###Filtramos con un pasabanda las frecuencias mayores a 300Hz y menores a 30Hz, y suavizamos picos

In [0]:
#bandpaseamos la señal entre 30-300 Hz
bb, ab = scsig.butter(4, [30.*2./fs, 300.*2./fs], btype='band')
strain_H1_bp = scsig.filtfilt(bb, ab, strain_H1)
strain_L1_bp = scsig.filtfilt(bb, ab, strain_L1)

fft_H1 = np.fft.fft(strain_H1_bp)
fft_L1 = np.fft.fft(strain_L1_bp)

#buscamos picos
peaks_H1 = scsig.find_peaks(fft_H1)
peaks_L1 = scsig.find_peaks(fft_L1)

#eliminamos picos en base a los encontrados
for i in range(len(peaks_H1)):
    fft_L1[i]=0
    fft_H1[i]=0

#recuperamos señal
sig_L1 = np.real(np.fft.ifft(fft_L1))
sig_H1 = np.real(np.fft.ifft(fft_H1))

###Calculamos máxima autocorrelación

In [0]:
#matriz de autocorrelación (función creada por Santiago Ivulich)

N = len(sig_L1)
L = int(np.floor(200*1e-3*fs))
K = int(np.floor((N-1)*2./L+1))
D = L//2
segments_H1 = [sig_H1[i*D:i*D+L] for i in range(K)]
segments_L1 = [sig_L1[i*D:i*D+L] for i in range(K)]

MAX_CORR = 0
MAX_POS = [0,0]
MAX_TAU = 0
for i,seg_H in enumerate(segments_H1):
    for j,seg_L in enumerate(segments_L1):
        if(i>10 and j>10 and i<K-10 and j<K-10):
            corr_ij = np.correlate(seg_H, seg_L,"same")
            corr_ij_max = np.amax(np.abs(corr_ij))
            indx = np.where(np.abs(corr_ij)==corr_ij_max)
            if(corr_ij_max > np.abs(MAX_CORR)):
                MAX_CORR = corr_ij[indx[0][0]]
                MAX_POS = [i,j]
                MAX_TAU = (indx[0][0]-len(corr_ij)/2)/fs

plt.figure(figsize=(10,5))
plt.plot(np.linspace(-0.1,0.1,len(segments_H1[MAX_POS[0]])),segments_H1[MAX_POS[0]],label='H1 strain')
plt.plot(np.linspace(-0.1,0.1,len(segments_H1[MAX_POS[0]])),np.sign(MAX_CORR)*np.roll(segments_L1[MAX_POS[1]],int(MAX_TAU*fs)),label='L1 strain')
plt.xlim((-0.1,0.1))
plt.legend(loc='lower left')
plt.show()
print('Max correlation with a diference of {}ms'.format(MAX_TAU*1000))

    

El error puede deberse al método utilizado para suavizar los picos.